In [10]:
# =========================
# Full IBM QAOA Ambulance Routing
# =========================

import json
import numpy as np
from itertools import permutations
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_algorithms import QAOA
from qiskit_algorithms.optimizers import COBYLA
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

# -------------------------
# 1) Load IBM token
# -------------------------
QiskitRuntimeService.save_account(
    token="o91rMR3O7WX5aSu7DT4J5XpJSQhVqpFXSiOZvKk72u2H",
    instance="crn:v1:bluemix:public:quantum-computing:us-east:a/4eb81c7e8955409ab06760d82ca1e020:77c7eae9-bf47-4300-a309-e6e18fb6b9c1::",
    region="us-east",
    set_as_default=True,
    overwrite=True
)

service = QiskitRuntimeService()

# -------------------------
# 2) Load data
# -------------------------
with open("OptimizationProblemData.json", "r") as f:
    data = json.load(f)

hospital = data["locations"]["hospital"]["coordinates"]
patients = data["locations"]["patients"]
n_patients = len(patients)
max_stops = 3  # max stops per trip

locations = [hospital] + [p["coordinates"] for p in patients]
n_locations = len(locations)

# Haversine distance
def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat, dlon = lat2-lat1, lon2-lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    return 2*R*np.arcsin(np.sqrt(a))

# Distance matrix
dist_matrix = np.zeros((n_locations, n_locations))
for i in range(n_locations):
    for j in range(n_locations):
        if i != j:
            ci, cj = locations[i], locations[j]
            dist_matrix[i, j] = haversine(ci["latitude"], ci["longitude"], cj["latitude"], cj["longitude"])

# -------------------------
# 3) Build Quadratic Program
# -------------------------
qp = QuadraticProgram(name="Ambulance_Routing")

# Binary variables: x_{i,t} = patient i in trip t
for i in range(n_patients):
    for t in range(2):
        qp.binary_var(name=f"x_{i}_{t}")

# Each patient visited exactly once
for i in range(n_patients):
    constraint_terms = {f"x_{i}_{t}": 1 for t in range(2)}
    qp.linear_constraint(linear=constraint_terms, sense="==", rhs=1, name=f"patient_{i}_once")

# Max patients per trip
for t in range(2):
    constraint_terms = {f"x_{i}_{t}": 1 for i in range(n_patients)}
    qp.linear_constraint(linear=constraint_terms, sense="<=", rhs=max_stops, name=f"trip_{t}_max")

# Objective function (approximate distances)
linear_terms = {}
quadratic_terms = {}

for t in range(2):
    for i in range(n_patients):
        linear_terms[f"x_{i}_{t}"] = dist_matrix[0, i+1]*0.5
    for i in range(n_patients):
        for j in range(i+1, n_patients):
            quadratic_terms[(f"x_{i}_{t}", f"x_{j}_{t}")] = dist_matrix[i+1, j+1]*0.3

qp.minimize(linear=linear_terms, quadratic=quadratic_terms)

# Convert to QUBO
converter = QuadraticProgramToQubo()
qubo = converter.convert(qp)

# -------------------------
# 4) Setup QAOA for IBM
# -------------------------
backend_name = "ibm_brisbane"  # choose a real backend
sampler_backend = service.least_busy()
sampler = Sampler(sampler_backend)

optimizer = COBYLA(maxiter=50)
qaoa = QAOA(sampler=sampler, optimizer=optimizer, reps=2)  # sampler is required

algorithm = MinimumEigenOptimizer(qaoa)

# -------------------------
# 5) Solve QUBO
# -------------------------
result = algorithm.solve(qubo)
print("\nOptimization result:")
print(result)

# -------------------------
# 6) Interpret solution
# -------------------------
trips = [[], []]
for i in range(n_patients):
    for t in range(2):
        var_name = f"x_{i}_{t}"
        if result.variables_dict.get(var_name, 0) > 0.5:
            trips[t].append(patients[i]["id"])

# Function to calculate trip distance
def calculate_trip_distance(patient_ids):
    if not patient_ids:
        return 0, []
    patient_indices = [i+1 for i,p in enumerate(patients) if p["id"] in patient_ids]
    min_distance = float('inf')
    best_order = []
    for order in permutations(patient_indices):
        dist = dist_matrix[0, order[0]] + sum(dist_matrix[order[i], order[i+1]] for i in range(len(order)-1)) + dist_matrix[order[-1], 0]
        if dist < min_distance:
            min_distance = dist
            best_order = [patients[idx-1]["id"] for idx in order]
    return min_distance, best_order

total_distance = 0
for t in range(2):
    if trips[t]:
        distance, order = calculate_trip_distance(trips[t])
        total_distance += distance
        print(f"Trip {t+1}: Hospital -> {' -> '.join(order)} -> Hospital (Distance: {distance:.2f} km)")
    else:
        print(f"Trip {t+1}: Empty")

print(f"\nTotal distance: {total_distance:.2f} km")


IBMInputValueError: 'The instruction QAOA on qubits (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13) is not supported by the target system. Circuits that do not match the target hardware definition are no longer supported after March 4, 2024. See the transpilation documentation (https://quantum.cloud.ibm.com/docs/guides/transpile) for instructions to transform circuits and the primitive examples (https://quantum.cloud.ibm.com/docs/guides/primitives-examples) to see this coupled with operator transformations.'